<a href="https://colab.research.google.com/github/unverciftci/JAX-Denemeleri/blob/master/JAX_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google JAX İle Hızlandırılmış Hesaplama ve Otomatik Türev Alma 

Not: Uygulamamızda GPU kullanacağız. Runtime kısmından "Change runtime type", oradan da GPU'yu seçin. 

JAX aslında NumPy kütüphanesinin GPU ve TPU gibi hızlandırıcılara
ve otomatik türev almaya uygun olarak geliştirilmişidir diyebiliriz. 

Otomatik türev alma ([Auotograd](https://github.com/HIPS/autograd) veya [Autodiff](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html)) sayesinde döngülerin, koşulların (if gibi) ve türevin türevinin türevi bile alınabilir.    

JAX, [XLA](https://github.com/HIPS/autograd) derleyicisini (compiler) kullanarak kodu GPU ve TPU hızlandırıcılarında çalıştırır. Derleme, gerekli kütüphanelerin çağrılması ile hali hazırda çalışır. Fakat siz de kendi fonksiyonlarınızı jit (just-in-time) derleyicisine tanıtabilirsiniz.

Derleme ve otomatik türev birleştirilebilir ve karmaşık algoritmalarda bile azami performans sağlanır.

Şimdi gerekli kütüphaneleri çağıralım.

In [0]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random

Matrislerin Çarpımı

Birazdan rastgele veri üreteceğiz. NumPy ile Jax rastegele sayıları farklı şekilde üretmektedir. Bizim için bunun önemi paralel hesaplamada ortaya çıkıyor. Ayrıntılar için JAX dökümantosyonuna bakın. 

Rastgele sayı oluşturmak için bir anahtar (key) tanımlıyoruz. Buradan yarı rastgele sayı üretebiliriz (PRNG). 

In [0]:
key = random.PRNGKey(0)

Yukarıda bir şekilde rastgele sayıların her defasında aynı çıkması için kök (seed) oluşturduk (parantez içindeki 0 yerine başka sayı alarak bu değiştirilebilir).

Mesela bileşenleri normal dağılımdan oluşturulmuş 10 boyutlu vektör olan x:

In [0]:
x = random.normal(key, (10,))
print(x)

[-0.3721109   0.26423115 -0.18252768 -0.7368197  -0.44030377 -0.1521442
 -0.67135346 -0.5908641   0.73168886  0.5673026 ]


Büyük matrisleri çarpalım.

In [0]:
size = 3000
x = random.normal(key, (size, size), dtype=np.float32)
%time y = np.dot(x, x.T)
%time y = np.dot(x, x.T).block_until_ready()

CPU times: user 1.07 ms, sys: 263 µs, total: 1.33 ms
Wall time: 1.59 ms
CPU times: user 17.9 ms, sys: 16.8 ms, total: 34.7 ms
Wall time: 37.1 ms


block_until_ready dememizin sebebi şu: JAX normalde işlemi sonlandırmıyor sadece hazır hale getiriyor, eğer çıktı isterseniz veya başka yerde kullanılacaksa işleme devam ediyor. Bu şekilde işlemin tamamının süresin öğrendik.



İsterseniz NumPy ile karşılaştıralım.

In [0]:
import numpy as onp
%time y = onp.dot(x, x.T)

CPU times: user 1.63 s, sys: 5.5 ms, total: 1.64 s
Wall time: 874 ms
